In [1]:
import cv2
import numpy as np
import boltiot
import time

# Load the YOLO model
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

# Load the class names
with open('classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize Bolt device for sound production
api_key = "ef0a5424-11af-4d65-9108-c35168915817"
device_id = "BOLT989913"
mybolt = boltiot.Bolt(api_key, device_id)
pin = "0"  # Change to the pin number connected to the piezo buzzer

# Function to play sound based on detected animal class
def play_sound(class_name):
    if class_name == '20: elephant':
        frequency = 1000  # Adjust frequency as needed for the elephant sound
        duration = 500  # Adjust duration in milliseconds
        mybolt.analogWrite(pin, 255)  # Set the pin to HIGH (255 is the maximum value)
        time.sleep(duration / 1000)  # Delay for the duration of the sound
        mybolt.analogWrite(pin, 0)  # Set the pin to LOW (turn off the sound)
    elif class_name == '15: cat':
        frequency = 2000  # Adjust frequency as needed for the monkey sound
        duration = 1500  # Adjust duration in milliseconds
        mybolt.analogWrite(pin, 255)  # Set the pin to HIGH (255 is the maximum value)
        time.sleep(duration / 1000)  # Delay for the duration of the sound
        mybolt.analogWrite(pin, 0)  # Set the pin to LOW (turn off the sound)
    elif class_name == '22: zebra':
        frequency = 3000  # Adjust frequency as needed for the wild boar sound
        duration = 1000  # Adjust duration in milliseconds
        mybolt.analogWrite(pin, 255)  # Set the pin to HIGH (255 is the maximum value)
        time.sleep(duration / 1000)  # Delay for the duration of the sound
        mybolt.analogWrite(pin, 0)  # Set the pin to LOW (turn off the sound)
    elif class_name == '21: bear':
        frequency = 4000  # Adjust frequency as needed for the bear sound
        duration = 2500  # Adjust duration in milliseconds
        mybolt.analogWrite(pin, 255)  # Set the pin to HIGH (255 is the maximum value)
        time.sleep(duration / 1000)  # Delay for the duration of the sound
        mybolt.analogWrite(pin, 0)  # Set the pin to LOW (turn off the sound)
    elif class_name == '16: dog':
        frequency = 5000  # Adjust frequency as needed for the leopard sound
        duration = 1800  # Adjust duration in milliseconds
        mybolt.analogWrite(pin, 255)  # Set the pin to HIGH (255 is the maximum value)
        time.sleep(duration / 1000)  # Delay for the duration of the sound
        mybolt.analogWrite(pin, 0)  # Set the pin to LOW (turn off the sound)
    else:
        return  # No sound for other classes

# Set up the webcam
cap = cv2.VideoCapture(0)  # 0 represents the default webcam

while True:
    # Read the frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Perform object detection
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Process the detections
    class_ids = []
    confidences = []
    boxes = []
    for detection in outs:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Set a confidence threshold
                # Get the bounding box coordinates
                center_x = int(obj[0] * frame.shape[1])
                center_y = int(obj[1] * frame.shape[0])
                width = int(obj[2] * frame.shape[1])
                height = int(obj[3] * frame.shape[0])

                # Calculate the top-left corner coordinates
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)

                # Store the class id, confidence, and box coordinates
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, width, height])

    # Apply non-maximum suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Display the detected objects and labels
    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            x, y, width, height = box[0], box[1], box[2], box[3]
            class_id = class_ids[i]
            confidence = confidences[i]

            # If the class_id is within the known class range, get the class name
            if class_id < len(classes):
                class_name = classes[class_id]
            else:
                class_name = "Unknown"

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            label = f'{class_name}: {confidence:.2f}'
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Play sound based on detected animal class
            play_sound(class_name)

    # Display the frame with detections
    cv2.imshow('Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()